In [2]:
import requests
import json
import pandas as pd
import sqlite3
from bs4 import BeautifulSoup
import pymongo
from pandasql import sqldf
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
pysqldf = lambda q: sqldf(q, globals())

In [4]:
conn = sqlite3.connect('database.sqlite')
c = conn.cursor()

In [5]:
match_data = pd.read_sql_query('''SELECT * 
                                 FROM Matches''', conn)
match_data.drop_duplicates(subset='Match_ID', keep='first',inplace=True)
match_data.head()

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1,D2,2009,2010-04-04,Oberhausen,Kaiserslautern,2,1,H
1,2,D2,2009,2009-11-01,Munich 1860,Kaiserslautern,0,1,A
2,3,D2,2009,2009-10-04,Frankfurt FSV,Kaiserslautern,1,1,D
3,4,D2,2009,2010-02-21,Frankfurt FSV,Karlsruhe,2,1,H
4,5,D2,2009,2009-12-06,Ahlen,Karlsruhe,1,3,A


## team names (from germany)

In [ ]:
# print(match_data['HomeTeam'].nunique())
# match_data['HomeTeam'].unique()
#print all teams in matches table including german and british teams


In [6]:
#getting the dataframe flatview_advanced to subset the teams from all to german teams only
germans_df = pd.read_sql_query('''SELECT * 
                                 FROM FlatView_Advanced''', conn)
teams_df = pd.DataFrame(data=list(germans_df.Unique_Team.unique()), columns=['team'])
len(teams_df)

57

In [7]:
q = """
        SELECT *
        FROM match_data m
        INNER JOIN teams_df t
        on m.HomeTeam = t.team
        WHERE Season = 2011;
        """
matches2011_df = pysqldf(q)
matches2011_df.drop(columns=['team'],inplace=True, index=1)
matches2011_df.shape

(611, 9)

In [16]:
q = """
        SELECT HomeTeam as Team, sum(FTHG)
        FROM matches2011_df
        GROUP BY HomeTeam;
        """
HT_df = pysqldf(q)
HT_df.head()

,Team,sum(FTHG)
0,Aachen,15
1,Augsburg,20
2,Bayern Munich,49
3,Bochum,23
4,Braunschweig,21


In [14]:
q = """
        SELECT AwayTeam as Team, sum(FTAG)
        FROM matches2011_df
        WHERE Season = 2011
        GROUP BY AwayTeam;
        """
AT_df = pysqldf(q)
AT_df.head()

,Team,sum(FTAG)
0,Aachen,15
1,Augsburg,16
2,Bayern Munich,26
3,Bochum,18
4,Braunschweig,16


In [19]:
total_goal_df = pd.merge(HT_df, AT_df, on='Team' )
total_goal_df['total_goals'] = total_goal_df['sum(FTHG)']+total_goal_df['sum(FTAG)']
# total_goal_df = total_goal_df.drop(['AwayTeam', "sum(FTHG)",'sum(FTAG)'], axis=1)
total_goal_df

,Team,sum(FTHG),sum(FTAG),total_goals
0,Aachen,15,15,30
1,Augsburg,20,16,36
2,Bayern Munich,49,26,75
3,Bochum,23,18,41
4,Braunschweig,21,16,37


## Create a function to retrieve the total goals for each team

In [ ]:
team = 'Aachen'
def teamtotalgoals(team):
    return total_goal_df.loc[combine['Team']==team]
teamtotalgoals(team)

## The total number of wins the team earned during the 2011 season

In [ ]:
def win_team(record):
    """ if FTHG > FTAG then return 'hometeam' name; else, just return 'awayteam' name."""
    if record['FTHG'] > record['FTAG']:
        return record['HomeTeam']
    elif record['FTHG'] < record['FTAG']:
        return record['AwayTeam']

In [ ]:
matches2011_df['Winner'] = matches2011_df.apply(win_team, axis=1)
matches2011_df.head()

In [ ]:
matches2011_df['Winner'].value_counts()

In [ ]:
num_wins_df = pd.DataFrame(data=list(matches2011_df['Winner'].value_counts()), columns=['num_wins'])
# num_wins_df

In [ ]:
def num_of_wins(team):
    return matches2011_df['Winner'].value_counts()[team]

In [ ]:
team = 'Dortmund'
num_of_wins('Dortmund')

## A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)

In [ ]:
def wins_loses(df, team):
    wins = 0
    loses = 0
    for i , row in df.iterrows():
        if team == row['HomeTeam']:
            if row['FTR'] == 'H':
                wins += 1
            if row['FTR'] == 'A':
                loses += 1
        if team == row['AwayTeam']:
            if row['FTR'] == 'A':
                wins += 1
            if row['FTR'] == 'H':
                loses += 1
    return (wins, loses)

In [ ]:
team = 'Nurnberg'
wins_loses(matches2011_df, team)

In [ ]:
def bar(df, team):
    
    win_num, lose_num = wins_loses(df, team)
    
    return plt.bar(['win','lost'],[win_num, lose_num])


In [ ]:
team = 'Nurnberg'
bar(matches2011_df, team)

## The team's win percentage on days where it was raining during games in the 2011 season

In [ ]:
def get_keys(path):
    with open(path) as f:
        return json.load(f)

keys = get_keys("/Users/Erica/.secrets/darkapi.json")
api_key = keys['api_key']

In [ ]:
list_of_dates = (pd.to_datetime(matches2011_df['Date']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
game_dates = list(set(list_of_dates))
latitude = 52.52
longtitude = 13.4050

def get_weather(game_dates):
    alldates = []
    for date in game_dates:
        url = f'''https://api.darksky.net/forecast/{api_key}/{latitude},{longtitude},{date}'''
        response = requests.get(url)
        alldates.append(response.json())
    return alldates

In [ ]:
allweather = get_weather(game_dates)

In [ ]:
allweather[0].keys()

In [ ]:
allweather[0]['currently']['icon']

In [ ]:
def rainy_days(allweather):
    rainydays = []
    for i in allweather:
        if i['currently']['icon'] == 'rain':
            rainydays.append(i['currently']['time'])
    return rainydays

rain_dates = rainy_days(allweather)

In [ ]:
rain_dates_df = pd.DataFrame(rain_date, columns = ['timestamp_date'])
rain_dates_df

In [ ]:
list_of_dates = (pd.to_datetime(matches2011_df['Date']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
timestamp_tc = list(list_of_dates)
df2 = pd.DataFrame(timestamp_tc, columns = ['timestamp_all'])
matches2011_df['timestamp_all'] = df2
matches2011_df

In [ ]:
matches2011_df['timestamp_all'] = rainy_df
matches2011_df.head()

In [ ]:
merged=matches2011_df.merge(rain_dates_df, left_on='timestamp_all', right_on='timestamp_date')

In [ ]:
def winpercentage(team):
    totalAwayTeam= 0
    totalHomeTeam =0
    totalgames = 0
    if merged.loc[merged['Winner'] == team].shape[0]!=0:
        totalwins = merged['Winner'].value_counts()[team]
    if merged.loc[merged['HomeTeam'] == team].shape[0]!=0:
        totalHomeTeam = merged['HomeTeam'].value_counts()[team]
    if merged.loc[merged['AwayTeam'] == team].shape[0]!=0:
        totalAwayTeam= merged['AwayTeam'].value_counts()[team]
    totalgames= totalHomeTeam+totalAwayTeam
    percentagewin= totalwins*100/totalgames
    return percentagewin

In [ ]:
team = 'Paderborn'
winpercentage(team)